# Analysis of the returns of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from statsmodels.tsa.seasonal import STL

In [ ]:
# Get bitcoin price df and set date as index, for more efficiency and simplicity
df_btc = pd.read_csv("../data/BTC.csv", index_col="date", parse_dates=True)

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get OHLC average as price
df_btc["price"] = df_btc[["open", "high", "low", "close"]].mean(axis=1)
df_btc.drop(columns=["open", "high", "low", "close"], inplace=True)

## Price change year-over-year

In [ ]:
# Get YoY returns
# Get yearly bitcoin price df with first and last prices
df_btc_yearly = df_btc.groupby(df_btc.index.year)["price"].agg(
    first_price="first",
    last_price="last"
)
# Get YoY return
df_btc_yearly["price_change"] = (df_btc_yearly["last_price"] - df_btc_yearly["first_price"]) / df_btc_yearly["first_price"]

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly, x=df_btc_yearly.index, y="price_change", color="#f7931a")

plt.title("Year-over-year return of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

#plt.yscale("symlog")

save_chart_as_png("3_BTC_YoY_return")

In [ ]:
# Highest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmax()]]

In [ ]:
# Lowest YoY return
df_btc_yearly.loc[[df_btc_yearly["price_change"].idxmin()]]

In [ ]:
# Average YoY return
df_btc_yearly["price_change"].mean().round(4)

In [ ]:
# Median YoY return
round(df_btc_yearly["price_change"].median(), 4)

In [ ]:
# Standard deviation YoY return
round(df_btc_yearly["price_change"].std(), 4)

**Key takeaways:**
- ...

## Cumulative returns

In [ ]:
# Cumulative product YoY return
(1 + df_btc_yearly["price_change"]).cumprod().iloc[-1].round(4) - 1